In [1]:
# перезалито

import string
import pymorphy2
import math

translator = str.maketrans('', '', string.punctuation)
morph = pymorphy2.MorphAnalyzer()

In [2]:
s = open("C:/Users/1/Desktop/corp.txt", "r", encoding="utf-8")
s = s.read()

In [3]:
stop_words = ['а', 'бы', 'в', 'во', 'вот', 'для', 'до', 'если', 'же', 'за', 'и', 'из', 'или', 'к', 'ко', 'на', 'над', 'но', 'о', 'об', 'от', 'по', 'при', 'с', 'то', 'у', 'чтобы', 'да', 'нет', 'не', 'ни', '—', 'это', 'много', 'немного', 'мало', 'немало', 'больше', 'меньше', 'я', 'мы', 'ты', 'вы', 'он', 'она', 'оно', 'они', 'можно', 'мочь', 'нельзя', 'надо', 'нужно', 'иметь', 'быть', 'который', 'какой', 'как', 'что', 'где', 'когда', 'потому', 'только', 'также', 'тоже']

In [4]:
words = s.split()

words_new = []

for word in words:
    word = word.translate(translator)
    word = word.lower()
    words_new.append(word)
words = words_new

In [5]:
dic_freq = {}

def add_to_dic(word, dic_freq):
    if word not in dic_freq:
        dic_freq[word] = 1
    else:
        dic_freq[word] +=1
    return dic_freq

for i in range (len(words)-1):
    if words[i] == "google":
        words[i+1] = morph.parse(words[i+1])[0].normal_form
        words[i-1] = morph.parse(words[i-1])[0].normal_form
        if words[i+1] not in stop_words and words[i+1]:
            bigr1 = words[i] + " " + words[i+1]
            add_to_dic(bigr1, dic_freq)      
        if words[i-1] not in stop_words:
            bigr2 = words[i-1] + " " + words[i]
            add_to_dic(bigr2, dic_freq)

In [6]:
f = open("C:/Users/1/Desktop/google_ngrams.csv", "a", encoding="utf-8")
f.write("word1" + ";" + "word2" + ";" + "count1" + ";" + "count2" + ";" + "count_bigr" + ";" + "pmi" + "\n")
f.close()

In [7]:
f = open("C:/Users/1/Desktop/google_ngrams.csv", "a", encoding="utf-8")
del dic_freq[' google']
for key, value in dic_freq.items():
    if value > 30:
        key_words = key.split()
        count1 = words.count(key_words[0])
        count2 = words.count(key_words[1])
        f.write(key_words[0] + ";" + key_words[1] + ";" + str(count1) + ";" + str(count2) + ";" + str(value) + "\n")
f.close()

In [8]:
dic_pmi = {}
num_w = len(words) 

import csv
with open("C:/Users/1/Desktop/google_ngrams.csv") as csvin:
    readfile = csv.reader(csvin, delimiter=';')
    next(readfile)
    row0 = list(readfile)
    with open("C:/Users/1/Desktop/google_ngrams_pmi.tsv", 'w') as csvout:
        writefile = csv.writer(csvout, delimiter='\t', lineterminator='\n')
        for row in row0:
            count1 = words.count(row[0])
            count2 = words.count(row[1])
            p_w1 = count1 / float(num_w)
            p_w2 = count2 / float(num_w)
            p_w1_w2 = float(row[4]) / float(num_w -1)
            if p_w1 != 0 and p_w2 != 0:
                pmi = math.log(p_w1_w2 /float(p_w1 * p_w2))
                dic_pmi[row[0] + " " + row[1]] = pmi
                row.append(pmi)
                writefile.writerow(row)

In [9]:
# топ-10 по частотности: частотные слова (компания, год), топ-10 по pmi: слова, в основном употреблюящиеся только с "google"

def top_ten(dic):
    dic_sorted_keys = sorted(dic, key=dic.get, reverse=True)[:10]
    for key in dic_sorted_keys:
        print(key, dic[key])
    
top_ten(dic_freq)
print ("\n")
top_ten(dic_pmi)

компания google 292
google chrome 201
google maps 120
google play 117
год google 110
система google 95
google android 84
карта google 83
сервис google 80
google earth 74


google talk 5.434883604019757
google play 5.373460791393351
google io 5.305154497344973
google earth 5.28940086167455
google maps 5.272364674521982
google chrome 5.246580557366268
google wave 5.183569175738851
google code 4.94100753856754
google android 3.5451437264314984
google inc 3.4116123338069757
